In [1]:
import sc2
from sc2 import run_game, maps, Race, Difficulty
from sc2.player import Bot, Computer
from sc2.constants import NEXUS, PROBE, PYLON, ASSIMILATOR, GATEWAY, CYBERNETICSCORE, STALKER, STARGATE, VOIDRAY
import random

class Sentdebot(sc2.BotAI):
    def __init__(self):
        self.ITERATIONS_PER_MINUTE = 165
        self.MAX_WORKERS = 65
        
    async def on_step(self, iteration):
        self.iteration = iteration
        await self.distribute_workers()
        await self.build_workers()
        await self.build_pylons()
        await self.build_assimilators()
        await self.expand()
        await self.barracks()
        await self.offencive_force()
        await self.attack()
        
    async def build_workers(self):
        if len(self.units(NEXUS))*16 > len(self.units(PROBE)):
            if len(self.units(PROBE)) < self.MAX_WORKERS:
                for nexus in self.units(NEXUS).ready.idle:
                    if self.can_afford(PROBE):
                        await self.do(nexus.train(PROBE))
                
    async def build_pylons(self):
        if self.supply_left < 5 and not self.already_pending(PYLON):
            nexuses = self.units(NEXUS).ready
            if nexuses.exists:
                if self.can_afford(PYLON):
                    await self.build(PYLON, near=nexuses.first)
                    
    async def build_assimilators(self):
        for nexus in self.units(NEXUS).ready:
            vespenes = self.state.vespene_geyser.closer_than(15.0, nexus)
            for vespene in vespenes:
                if not self.can_afford(ASSIMILATOR):
                    break
                worker = self.select_build_worker(vespene.position)
                if worker is None:
                    break
                if not self.units(ASSIMILATOR).closer_than(1.0, vespene).exists:
                    await self.do(worker.build(ASSIMILATOR, vespene))
                    
    async def expand(self):
        if self.units(NEXUS).amount < 3 and self.can_afford(NEXUS):
            await self.expand_now()
            
    async def barracks(self):
        if self.units(PYLON).ready.exists:
            pylon = self.units(PYLON).ready.random

            if self.units(GATEWAY).ready.exists and not self.units(CYBERNETICSCORE):
                if self.can_afford(CYBERNETICSCORE) and not self.already_pending(CYBERNETICSCORE):
                    await self.build(CYBERNETICSCORE, near=pylon)

            elif len(self.units(GATEWAY)) < ((self.iteration / self.ITERATIONS_PER_MINUTE)/2):
                if self.can_afford(GATEWAY) and not self.already_pending(GATEWAY):
                    await self.build(GATEWAY, near=pylon)

            if self.units(CYBERNETICSCORE).ready.exists:
                if len(self.units(STARGATE)) < ((self.iteration / self.ITERATIONS_PER_MINUTE)/2):
                    if self.can_afford(STARGATE) and not self.already_pending(STARGATE):
                        await self.build(STARGATE, near=pylon)
                    
    async def offencive_force(self):
        for gw in self.units(GATEWAY).ready.noqueue:
            if not self.units(STALKER).amount > self.units(VOIDRAY).amount:

                if self.can_afford(STALKER) and self.supply_left > 0:
                    await self.do(gw.train(STALKER))

        for sg in self.units(STARGATE).ready.noqueue:
            if self.can_afford(VOIDRAY) and self.supply_left > 0:
                await self.do(sg.train(VOIDRAY))
                
    def find_target(self, state):
        if len(self.known_enemy_units) > 0:
            return random.choice(self.known_enemy_units)
        elif len(self.known_enemy_structures) > 0:
            return random.choice(self.known_enemy_structures)
        else:
            return self.enemy_start_locations[0]
                
    async def attack(self):
        aggressive_units = {STALKER: [15, 5],VOIDRAY: [8, 3]}

        for UNIT in aggressive_units:
            if self.units(UNIT).amount > aggressive_units[UNIT][0] and self.units(UNIT).amount > aggressive_units[UNIT][1]:
                for s in self.units(UNIT).idle:
                    await self.do(s.attack(self.find_target(self.state)))

            elif self.units(UNIT).amount > aggressive_units[UNIT][1]:
                if len(self.known_enemy_units) > 0:
                    for s in self.units(UNIT).idle:
                        await self.do(s.attack(random.choice(self.known_enemy_units)))
      
        
run_game(maps.get("AbyssalReefLE"), [Bot(Race.Protoss, Sentdebot()), Computer(Race.Terran, Difficulty.Hard)], realtime=False)
                
                

RuntimeError: This event loop is already running

INFO:sc2.protocol:Client status changed to Status.launched (was None)
INFO:sc2.controller:Creating new game
INFO:sc2.controller:Map:     AbyssalReefLE
INFO:sc2.controller:Players: Bot(Race.Protoss, <__main__.Sentdebot object at 0x0000019F09F3B788>), Computer Hard(Terran, RandomBuild)
INFO:sc2.protocol:Client status changed to Status.init_game (was Status.launched)
INFO:sc2.protocol:Client status changed to Status.in_game (was None)
INFO:root:Player 1 - Bot(Race.Protoss, <__main__.Sentdebot object at 0x0000019F09F3B788>)


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:71: DeprecationWarning: noqueue will be removed soon, please use idle instead


ERROR:sc2.bot_ai:Error: [<ActionResult.NotSupported: 2>] (action: UnitCommand(AbilityId.GATEWAYTRAIN_STALKER, Unit(name='Gateway', tag=4356571137), None, False))
ERROR:sc2.bot_ai:Error: [<ActionResult.NotSupported: 2>] (action: UnitCommand(AbilityId.GATEWAYTRAIN_STALKER, Unit(name='Gateway', tag=4356571137), None, False))
ERROR:sc2.bot_ai:Error: [<ActionResult.NotSupported: 2>] (action: UnitCommand(AbilityId.GATEWAYTRAIN_STALKER, Unit(name='Gateway', tag=4356571137), None, False))
ERROR:sc2.bot_ai:Error: [<ActionResult.NotSupported: 2>] (action: UnitCommand(AbilityId.GATEWAYTRAIN_STALKER, Unit(name='Gateway', tag=4356571137), None, False))
ERROR:sc2.bot_ai:Error: [<ActionResult.NotSupported: 2>] (action: UnitCommand(AbilityId.GATEWAYTRAIN_STALKER, Unit(name='Gateway', tag=4356571137), None, False))
ERROR:sc2.bot_ai:Error: [<ActionResult.NotSupported: 2>] (action: UnitCommand(AbilityId.GATEWAYTRAIN_STALKER, Unit(name='Gateway', tag=4356571137), None, False))
ERROR:sc2.bot_ai:Error: [<Ac